In [1]:
import os
import re
import glob
import numpy as np
import tifffile
import pandas as pd
from skimage.measure import regionprops, label
from tqdm import tqdm

In [2]:
# === Parameters ===
masks_folder = r"D:\Data\FK_P001_EX005_2025_03_21 60xWIA 1.5x CRISPRi mCh\TIF\results\stacked\cyto3_gpu_results"
stacks_folder = r"D:\Data\FK_P001_EX005_2025_03_21 60xWIA 1.5x CRISPRi mCh\TIF\results\stacked\aligned_stacks"

In [3]:
# === Utility: shape features ===
def shape_features(region):
    area = region.area
    perimeter = region.perimeter if region.perimeter > 0 else 1
    roundness = 4 * np.pi * area / (perimeter ** 2)
    major_axis = region.major_axis_length
    minor_axis = region.minor_axis_length
    aspect_ratio = major_axis / minor_axis if minor_axis != 0 else 0
    return {
        "Area": area,
        "Length": major_axis,
        "Width": minor_axis,
        "Roundness": roundness,
        "Aspect Ratio": aspect_ratio
    }

In [4]:
# === Main Loop ===
mask_files = [f for f in os.listdir(masks_folder) if f.endswith("_masks.tif")]
print(f"Found {len(mask_files)} mask files to process.")

# Flag for DataFrame structure output
first_file_processed = False

with tqdm(total=len(mask_files), desc="Measuring masks", dynamic_ncols=True) as pbar:
    for mask_fname in mask_files:
        mask_path = os.path.join(masks_folder, mask_fname)

        # Extract identifiers from mask filename
        match = re.search(r"Well([A-Z]\d+)_Seq(\d+)_Z(\d+)_slice\d+_stack_masks", mask_fname)
        if not match:
            print(f"⚠️ Could not parse identifiers from {mask_fname}")
            pbar.update(1)
            continue

        well, seq, z = match.groups()
        well_id = f"Well{well}"
        seq_id = f"Seq{seq}"
        z_id = f"Z{z}"

        # Debugging: Print extracted identifiers
        print(f"Extracted: Well={well_id}, Seq={seq_id}, Z={z_id}")

        stack_pattern = f"{well_id}_{seq_id}_{z_id}_*_stack.tif*"
        stack_search_path = os.path.join(stacks_folder, stack_pattern)
        stack_files = glob.glob(stack_search_path)

        # Debugging: Print the search path
        print(f"Searching for stack files: {stack_search_path}")

        if not stack_files:
            print(f"⚠️ Stack not found for {mask_fname}")
            pbar.update(1)
            continue

        stack_path = stack_files[0]

        # Read images
        stack = tifffile.imread(stack_path)
        masks = tifffile.imread(mask_path)
        fluo_img = stack[1]

        # Label masks and get regions
        labeled = label(masks)
        props = regionprops(labeled, intensity_image=fluo_img)

        data = []
        # Process individual regions
        for prop in props:
            features = shape_features(prop)
            data.append({
                "Label": prop.label,
                "Area": features["Area"],
                "Length": features["Length"],
                "Width": features["Width"],
                "Roundness": features["Roundness"],
                "Aspect Ratio": features["Aspect Ratio"],
                "Mean Intensity": prop.mean_intensity,
                "Centroid X": prop.centroid[1],
                "Centroid Y": prop.centroid[0],
                "Type": "Object"
            })

        # Add background measurements
        background_mask = (labeled == 0)
        background_pixels = fluo_img[background_mask]
        if background_pixels.size > 0:
            data.append({
                "Label": 0,
                "Area": np.sum(background_mask),
                "Length": np.nan,
                "Width": np.nan,
                "Roundness": np.nan,
                "Aspect Ratio": np.nan,
                "Mean Intensity": np.mean(background_pixels),
                "Centroid X": np.nan,
                "Centroid Y": np.nan,
                "Type": "Background"
            })

        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Show DataFrame structure once
        if not first_file_processed:
            print("\nDataFrame structure:")
            print(df.dtypes)
            # Ensure DataFrame is not empty before printing
            if not df.empty:
                print("\nExample row:")
                print(df.iloc[0])
            else:
                print("\nDataFrame is empty for the first file.")
            first_file_processed = True

        # Save results
        base_name = os.path.splitext(mask_fname)[0].replace("_masks", "")
        excel_output_path = os.path.join(masks_folder, f"{base_name}.xlsx")
        df.to_excel(excel_output_path, index=False)

        pbar.update(1)

print("✅ Measurement extraction complete.")

Found 389 mask files to process.


Measuring masks:   0%|                                                                         | 0/389 [00:00<?, ?it/s]

Extracted: Well=WellA01, Seq=Seq0000, Z=Z10
Searching for stack files: D:\Data\FK_P001_EX005_2025_03_21 60xWIA 1.5x CRISPRi mCh\TIF\results\stacked\aligned_stacks\WellA01_Seq0000_Z10_*_stack.tif*


ValueError: Label and intensity image shapes must match, except for channel (last) axis.

In [5]:
print("labeled shape:", labeled.shape)
print("fluo_img shape:", fluo_img.shape)

labeled shape: (3200, 3200)
fluo_img shape: (3000, 3000)
